In [1]:
import numpy as np
import matplotlib.pyplot as plt

from proj1_helpers import *
from functions import *

y, x, ids = load_csv_data('train.csv')

In [2]:
columns_to_keep = [0,1,2,3,4,5,6,10,11,12,13,22,24]
x1 = x[:, columns_to_keep]
y1=y
#remove -999 and put them clos to the rest of the data
min_ = x1[x1!=-999].min()-1
x1[x1<=-998]= min_


In [3]:
# remove outliers
x1 = x1
y1 = y1
threshold = []
for i in range(x1.shape[1]):
    # get value of each column of the 99 percentile
    threshold += [np.percentile(x1[:,i], 99)]
for i in range(x1.shape[1]):
    y1 = y1[x1[:,i]<=threshold[i]]
    x1 = x1[x1[:,i]<=threshold[i]]
print('{} % of the data was removed'.format(int(100*(1-len(x1)/len(x)))))

8 % of the data was removed


In [38]:
x_train, x_test, y_train, y_test = split_data(x1, y1, ratio=0.8, seed =2)


#set to 0 instead of -1
y_train[y_train<0]=0
y_test[y_test<0]=0
y_test = y_test.reshape(-1,1)
y_train = y_train.reshape(-1,1)

x_train, mean_x, std_x= standardize(x_train)
x_test = (x_test-mean_x)/std_x
degree=2
lambda_=0
x_train = build_poly(x_train, degree)
x_test = build_poly(x_test, degree)

initial_w = np.zeros([x_train.shape[1],1])

w, loss = stochastic_gradient_descent(y_train, x_train, initial_w, batch_size=10, max_iters=5000, gamma=0.0001,\
                                      lambda_1=0, lambda_2=0, compute_gradient=grad_logistic, compute_loss = compute_logi_loss)

In [37]:
#find optimal threshold
for i in range(0,11):
    z = sigmoid(x_test.dot(w[-1]))
    z[z>i/10] = 1
    z[z<i/10] = 0
    print('thre: {}, accuracy {}'.format(i/10, np.mean(z== y_test)))

thre: 0.0, accuracy 0.3312210099954269
thre: 0.1, accuracy 0.34271901744299993
thre: 0.2, accuracy 0.37046231571612115
thre: 0.3, accuracy 0.5165174974412579
thre: 0.4, accuracy 0.7031205766424947
thre: 0.5, accuracy 0.7030116939962109
thre: 0.6, accuracy 0.6811916116809303
thre: 0.7, accuracy 0.6737658152043727
thre: 0.8, accuracy 0.6699549225844385
thre: 0.9, accuracy 0.6688007665338298
thre: 1.0, accuracy 0.6687789900045731


In [10]:
y_t, x_t, ids = load_csv_data('test.csv')

In [15]:
x_t1 = x_t[:,columns_to_keep]
x_t1[x_t1==-999]= min_

x_t2 = (x_t1-mean_x)/std_x
x_t2 = build_poly(x_t2, degree=degree)

z = sigmoid(x_t2.dot(w[-1]))
z[z>0.5] = 1
z[z<=0.5] = -1
y_p = z

In [17]:
create_csv_submission(ids, y_p, 'prediction_logistic_regression.csv')

In [13]:
# 0.76459 on kaggle